Preprocessing

In [1]:
import os
from dotenv import load_dotenv

# Load các biến môi trường từ tệp .env
load_dotenv('enviroment.env')

# Lấy giá trị của các biến môi trường
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')
QDRANT_URL = os.getenv('QDRANT_URL')
EMBEDDING_MODEL_NAME = 'dangvantuan/vietnamese-embedding'

In [2]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader,TextLoader
# Load file dữ liệu sổ tay sinh viên đã được chuẩn hóa
loader_sotay = TextLoader("data/combine/data.txt", 'utf-8')

In [3]:
print(loader_sotay.load())

[Document(metadata={'source': 'data/combine/data.txt'}, page_content='1. Học bổng khuyến khích học tập (KKHT)\nNăm học 2023-2024 ĐHBK Hà Nội dành khoảng 70 tỷ đồng cho quỹ học bổng KKHT để xét cấp cho sinh viên có kết quả học tập và rèn luyện tốt. Học bổng KKHT được xét theo học kỳ và có 3 mức:\n\nHọc bổng loại khá (loại C): Tương đương với mức học phí sinh viên phải đóng\nHọc bổng loại giỏi (loại B): Bằng 1,2 lần học bổng loại khá.\nHọc bổng loại suất sắc (loại A): Bằng 1,5 lần học bổng loại khá.\n\nĐiều kiện được xét, cấp học bổng KKHT:\n\nHọc bổng loại khá: Sinh viên có kết quả học tập và rèn luyện đạt từ loại khá trở lên.\nHọc bổng loại giỏi: Sinh viên có kết quả học tập và rèn luyện đạt từ loại giỏi trở lên.\nHọc bổng loại xuất sắc: Sinh viên có kết quả học tập và rèn luyện đạt từ loại xuất sắc trở lên.\n\nHọc bổng KKHT được xét cấp dựa trên kết quả học tập và rèn luyện của sinh viên (sinh viên không cần phải đăng ký).\nChi tiết Quy định về việc xét cấp học bổng KKHT (áp dụng từ n

In [4]:
documents_sotay = loader_sotay.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                chunk_overlap=400)

texts_sotay = text_splitter.split_documents(documents_sotay)

In [5]:
len(texts_sotay)

92

In [6]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
embeddings = HuggingFaceBgeEmbeddings(model_name=EMBEDDING_MODEL_NAME)

In [7]:
# Encode dữ liệu text qua lớp embedding,
# rồi lưu lại trên vector database server
qdrant = Qdrant.from_documents(
    texts_sotay,
    embeddings,
    url=QDRANT_URL,
    prefer_grpc=False,
    collection_name="stsvHust_vector_db_v2",
    api_key=QDRANT_API_KEY,
)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Test tìm văn bản liên quan đến query
query = "Địa chỉ của phòng đào tạo"
found_docs = qdrant.similarity_search(query,k=15)

In [ ]:
found_docs


[Document(page_content='Phòng 103 nhà C1\nĐiện thoại: 024.38693108\nEmail: ctsv@hust.edu.vn\nThông báo: Kể từ ngày 21/9/2023 có sự thay đổi về tên gọi và chức năng các đơn vị, theo đó Phòng Đào tạo sẽ chuyển thành Ban Đào tạo, tuy nhiên địa chỉ phòng làm việc không thay đổi.\n------------------------------------------------------------------------\nBan Đào tạo tiếp nhận các đề xuất, thắc mắc, phản hồi của sinh viên qua thư điện tử (email), hạn chế tối đa việc sinh viên phải lên gặp trực tiếp. Việc sử dụng email khi trao đổi sẽ mang lại các lợi ích sau:', metadata={'source': 'data/combine/data.txt'}),
 Document(page_content='26. \xa0\n\n\nChuyển tiếp từ CN lên kỹ sư.\n\n\n[CN lên KS]\n\n\nhue.nguyenkim@hust.edu.vn\nChuyên viên Nguyễn Kim Huệ (202A-C1)\n\n\nTải tại đây\n\n\n\n\n27. \xa0\n\n\nChuyển ngành đào tạo/chương trình đào tạo\n\n\n[Chuyển ngành]\n\n\nhue.nguyenkim@hust.edu.vn\nChuyên viên Nguyễn Kim Huệ (202A-C1)\n\n\nTải tại đây\n\n\n\n\n28. \xa0\n\n\nChuyển sang học tại Trung tâ